In [1]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
### SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]
SCOPES = ["https://www.googleapis.com/auth/calendar"] ## read and write scope

In [3]:
"""Shows basic usage of the Google Calendar API.
Prints the start and name of the next 10 events on the user's calendar.
  """
creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])

except HttpError as error:
    print(f"An error occurred: {error}")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=437371744632-iqam1opjo9ica41nudi1du5bv1tu0dii.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A61906%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=ya9s5ZVUw2UmBKUKYkZattNjyWKiV5&access_type=offline
Getting the upcoming 10 events
2025-03-24T14:00:00+03:00 Progress meeting
2025-03-26T14:00:00+03:00 Progress meeting
2025-03-28T14:00:00+03:00 Progress meeting
2025-03-31T14:00:00+03:00 Progress meeting
2025-04-02T14:00:00+03:00 Progress meeting
2025-04-04T14:00:00+03:00 Progress meeting
2025-04-07T14:00:00+03:00 Progress meeting
2025-04-09T14:00:00+03:00 Progress meeting
2025-04-11T14:00:00+03:00 Progress meeting
2025-04-14T14:00:00+03:00 Progress meeting


In [4]:
service.calendarList().list().execute()

{'kind': 'calendar#calendarList',
 'etag': '"p33fsbd5onieoo0o"',
 'nextSyncToken': 'CN_FtLi8nYwDEhBlZ2VAbmFub21pbmRzLmFp',
 'items': [{'kind': 'calendar#calendarListEntry',
   'etag': '"1741983107837608"',
   'id': 'ege@nanominds.ai',
   'summary': 'ege@nanominds.ai',
   'timeZone': 'Europe/Istanbul',
   'colorId': '14',
   'backgroundColor': '#9fe1e7',
   'foregroundColor': '#000000',
   'selected': True,
   'accessRole': 'owner',
   'defaultReminders': [{'method': 'popup', 'minutes': 10}],
   'notificationSettings': {'notifications': [{'type': 'eventCreation',
      'method': 'email'},
     {'type': 'eventChange', 'method': 'email'},
     {'type': 'eventCancellation', 'method': 'email'},
     {'type': 'eventResponse', 'method': 'email'}]},
   'primary': True,
   'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}},
  {'kind': 'calendar#calendarListEntry',
   'etag': '"1742565729978927"',
   'id': 'en.turkish#holiday@group.v.calendar.google.com',
   'summary':

In [5]:
service.calendarList().get(calendarId='ege@nanominds.ai').execute()
### service.calendarList().get(calendarId='primary').execute(): same for now

{'kind': 'calendar#calendarListEntry',
 'etag': '"1741983107837608"',
 'id': 'ege@nanominds.ai',
 'summary': 'ege@nanominds.ai',
 'timeZone': 'Europe/Istanbul',
 'colorId': '14',
 'backgroundColor': '#9fe1e7',
 'foregroundColor': '#000000',
 'selected': True,
 'accessRole': 'owner',
 'defaultReminders': [{'method': 'popup', 'minutes': 10}],
 'notificationSettings': {'notifications': [{'type': 'eventCreation',
    'method': 'email'},
   {'type': 'eventChange', 'method': 'email'},
   {'type': 'eventCancellation', 'method': 'email'},
   {'type': 'eventResponse', 'method': 'email'}]},
 'primary': True,
 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}

In [10]:
page_token = None
while True:
  calendar_list = service.calendarList().list(pageToken=page_token).execute()
  for calendar_list_entry in calendar_list['items']:
    print(calendar_list_entry['summary'])
  page_token = calendar_list.get('nextPageToken')
  if not page_token:
    break

ege@nanominds.ai
Holidays in Turkey


In [8]:
event = {
  'summary': 'Google I/O 2015',
  'location': '800 Howard St., San Francisco, CA 94103',
  'description': 'A chance to hear more about Google\'s developer products.',
  'start': {
    'dateTime': '2015-05-28T09:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'end': {
    'dateTime': '2015-05-28T17:00:00-07:00',
    'timeZone': 'America/Los_Angeles',
  },
  'recurrence': [
    'RRULE:FREQ=DAILY;COUNT=2'
  ],
  'attendees': [
    {'email': 'lpage@example.com'},
    {'email': 'sbrin@example.com'},
  ],
  'reminders': {
    'useDefault': False,
    'overrides': [
      {'method': 'email', 'minutes': 24 * 60},
      {'method': 'popup', 'minutes': 10},
    ],
  },
}

event = service.events().insert(calendarId='primary', body=event).execute()
print('Event created: %s' % (event.get('htmlLink')))
print(event['summary'], event['start']['dateTime'], event['end']['dateTime']) 

Event created: https://www.google.com/calendar/event?eid=anNrYnYwZWhxNHZ1M3BiZjFwOWtzMG10aDRfMjAxNTA1MjhUMTYwMDAwWiBlZ2VAbmFub21pbmRzLmFp
Google I/O 2015 2015-05-28T19:00:00+03:00 2015-05-29T03:00:00+03:00
